In [21]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
--2019-04-21 08:56:39--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.223, 91.189.89.222, 2001:67c:1560:8003::8003, ...
Connecting to launchpad.net (launchpad.net)|91.189.89.223|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-04-21 08:56:39 ERROR 404: Not Found.

dpkg: error: cannot access archive 'google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb': No such file or directory
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
··········


In [22]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [23]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import cv2
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from sklearn.datasets import load_files
import matplotlib.pyplot as plt

In [0]:
def load_data(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    return files

In [0]:
def ext_img(img):
    imag = cv2.imread(img)
    imag = cv2.cvtColor(imag,cv2.COLOR_BGR2GRAY)
    imag = imag/255
    imag = np.expand_dims(imag,axis=2)
    return imag

In [0]:
def tensor_4d(fil):
    lis = [ext_img(im) for im in tqdm(fil)]
    return np.stack(lis,axis=0)

In [0]:
fold_path = 'drive/datasets/shapes'

In [0]:
img_files = load_data(fold_path)

In [30]:
imgs = tensor_4d(img_files)

100%|██████████| 300/300 [00:00<00:00, 515.17it/s]


In [31]:
imgs.shape

(300, 28, 28, 1)

In [0]:
class Encoder(tf.keras.Model):
    
    def __init__(self):
        super(Encoder,self).__init__()
        self.conv8 = tf.keras.layers.Conv2D(8,3,activation=tf.nn.relu,strides=(1,1),padding='SAME')
        self.conv16 = tf.keras.layers.Conv2D(16,3,activation=tf.nn.relu,strides=(1,1),padding='SAME')
        self.conv32 = tf.keras.layers.Conv2D(32,3,activation=tf.nn.relu,strides=(1,1),padding='SAME')
        self.max_pool = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))
        self.gap = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))
        
    def call(self,inp):
        x = self.conv32(inp)
        print(x.shape)
        x = self.max_pool(x)
        print(x.shape)
        x = self.conv16(x)
        print(x.shape)
        x = self.max_pool(x)
        print(x.shape)
        x = self.conv8(x)
        print(x.shape)
        #x = self.gap(x)
        print(x.shape)
        
        
        return x
        
        
        

In [0]:
inp = tf.keras.Input(shape=(28,28,1))
enc = Encoder()

In [85]:
enc(inp)

(None, 28, 28, 32)
(None, 14, 14, 32)
(None, 14, 14, 16)
(None, 7, 7, 16)
(None, 7, 7, 8)
(None, 7, 7, 8)


<tf.Tensor 'encoder_22/conv2d_52/Relu:0' shape=(None, 7, 7, 8) dtype=float32>

In [0]:
class Decoder(tf.keras.Model):
    
    def __init__(self):
        super(Decoder,self).__init__()
        self.convtr8 = tf.keras.layers.Conv2DTranspose(8,3,strides=(1,1),padding='SAME',activation=tf.nn.relu)
        self.convtr16 = tf.keras.layers.Conv2DTranspose(16,3,strides=(1,1),padding='SAME',activation=tf.nn.relu)
        self.convtr32 = tf.keras.layers.Conv2DTranspose(32,3,strides=(1,1),padding='SAME',activation=tf.nn.relu)
        self.convtr1 = tf.keras.layers.Conv2DTranspose(1,3,strides=(1,1),padding='SAME',activation=tf.nn.relu)
        self.upsample = tf.keras.layers.UpSampling2D(size=(2,2))
        
    def call(self,inp):
        x = self.convtr8(inp)
        print(x.shape)
        x = self.upsample(x)
        print(x.shape)
        x = self.convtr16(x)
        print(x.shape)
        x = self.upsample(x)
        print(x.shape)
        x = self.convtr32(x)
        print(x.shape)
        x = self.convtr1(x)
        print(x.shape)
        
        return x
        
        

In [0]:
dec = Decoder()

In [94]:
op = enc(inp)
oc = dec(op)

(None, 28, 28, 32)
(None, 14, 14, 32)
(None, 14, 14, 16)
(None, 7, 7, 16)
(None, 7, 7, 8)
(None, 7, 7, 8)
(None, 7, 7, 8)
(None, 14, 14, 8)
(None, 14, 14, 16)
(None, 28, 28, 16)
(None, 28, 28, 32)
(None, 28, 28, 1)


In [0]:
def model(ips):
    ip = tf.keras.layers.Input(shape=(28,28,1))(ips)
    encode = Encoder()
    encoded = encode(ip)
    decode = Decoder()
    decoded = decode(encoded)
    return decoded
    

In [0]:
def loss(ips,ops):
    mse_loss = tf.keras.losses.MeanSquaredError()
    mse = mse_loss(ips,ops)
    return mse

In [0]:
def grad(ips):
    with tf.GradientTape as tape:
        mod = model(ips)
        err = loss(ips,ops)
        

In [100]:
enc.trainable_variables

[<tf.Variable 'encoder_22/conv2d_52/kernel:0' shape=(3, 3, 16, 8) dtype=float32, numpy=
 array([[[[ 0.12742229,  0.03755347, -0.08212682, ..., -0.01270533,
           -0.08791089,  0.0085922 ],
          [ 0.07557671, -0.08032171,  0.10875101, ..., -0.0324626 ,
           -0.11322384, -0.09440915],
          [-0.15232477, -0.15771386,  0.15317203, ..., -0.15331945,
            0.12957256, -0.06216093],
          ...,
          [-0.01317258, -0.07791369,  0.16320015, ..., -0.09850299,
           -0.07018809, -0.00141335],
          [ 0.00251456, -0.06832417,  0.06301761, ..., -0.12723753,
            0.00244887, -0.14848697],
          [ 0.01071417, -0.1652641 , -0.06093534, ..., -0.08361614,
            0.12257902,  0.09210615]],
 
         [[ 0.15207617, -0.02658327,  0.08611004, ..., -0.16416208,
           -0.00182657,  0.04309642],
          [ 0.12470104, -0.0146966 ,  0.02683978, ..., -0.07726288,
            0.11872785,  0.0826339 ],
          [-0.02241063, -0.00763603, -0.105813